In [1]:
import sys
import os
import numpy as np
from scipy.sparse import coo_matrix
import sqlite3
from os.path import join
import pandas as pd

In [52]:
sys.path.append('C:\\Users\\pedro.camargo\\Documents\\sourcecode\\aequilibrae')
from aequilibrae.paths import Graph, NetworkSkimming, SkimResults, PathResults, path_computation

In [3]:
fldr = 'C:/D/Projects/Vancouver/'
aeg_pth = 'C:/D/Projects/Vancouver/Network/graph.aeg'

In [13]:
g = Graph()
g.load_from_disk(aeg_pth)
g.prepare_graph(g.all_nodes)
g.set_graph(cost_field='timau', skim_fields=['length'],block_centroid_flows=False)

result = SkimResults()
result.prepare(g)

skm = NetworkSkimming(g, result)
skm.execute()

In [16]:
# We create the sparse_matrix that we will use to generate the routing graph
idx = result.skims.index[:]
mat = coo_matrix(result.skims.matrix['timau'])
mat = coo_matrix((mat.data, (idx[mat.row], idx[mat.col])), shape=(int(idx.max()) + 1, int(idx.max()) + 1))
timau = mat.toarray()

In [17]:
# We create the sparse_matrix that we will use to generate the routing graph
idx = result.skims.index[:]
mat = coo_matrix(result.skims.matrix['length'])
mat = coo_matrix((mat.data, (idx[mat.row], idx[mat.col])), shape=(int(idx.max()) + 1, int(idx.max()) + 1))
length = mat.toarray()

In [7]:
nodelyr = sqlite3.connect(os.path.join(fldr, "Network/final_layer.gpkg"))
nds = pd.read_sql_query("SELECT fid FROM nodes where park_and_ride=1", nodelyr).fid.tolist()


In [84]:
g = Graph()
g.load_from_disk(aeg_pth)
lid = g.network['link_id'].max() + 1
links = []
a1 = []
a2 = []
a3 = []
a4 = []
a5 = []

for i in nds:
    for j in nds:
        a1.append(lid)
        a2.append(i)
        a3.append(j)
        a4.append(length[i, j] / 1.5)
        a5.append(timau[i, j] / 1.5)
        
        links.append((lid, i, j, 1, lgt, lgt, timau, timau))
        lid += 1

        
new_links = np.zeros(len(links), dtype=g.network.dtype )
new_links['link_id'][:] = np.array(a1)[:]
new_links['a_node'][:] = np.array(a2)[:]
new_links['b_node'][:] = np.array(a3)[:]
new_links['direction'].fill(1)
new_links['length_ab'][:] = np.array(a4)[:]
new_links['length_ba'][:] = np.array(a4)[:]
new_links['timau_ab'][:] = np.array(a5)[:]
new_links['timau_ba'][:] = np.array(a5)[:]


In [85]:
g.network = np.hstack((new_links,g.network))
g.prepare_graph(g.centroids)
g.num_links = g.graph.shape[0]

In [86]:
g.set_graph(cost_field='timau', skim_fields=[],block_centroid_flows=False)

# result = SkimResults()
# result.prepare(g)

# skm = NetworkSkimming(g, result)
# skm.execute()

True

In [87]:
r = PathResults()
r.prepare(g)

In [94]:
%%time
nds = np.array(nds)
hns = []
for i in g.centroids:
    path_computation(i, i, g, r)
    for j in g.centroids:
        r.update_trace(g, int(j))
        
        nzr = np.nonzero(np.in1d(r.path_nodes, nds))[0]
        if nzr.shape[0] >= 2:
            if timau[nzr[0], nzr[-1]] > r.milepost[-1] * 0.75:
                nzr = r.path_nodes[nzr]
                hns.append([i, j, nzr[0], nzr[-1]])

Wall time: 5min 34s


In [95]:
len(hns)

1019126

In [83]:
len(g.centroids)**2

3031081

In [93]:
len(hns)

2430632

In [ ]:
from aequilibrae import reserved_fields

k = Graph()

all_types = [k._Graph__integer_type, 
             k._Graph__integer_type, 
             k._Graph__integer_type, 
             np.int8,
             k._Graph__float_type,
             k._Graph__float_type]

all_titles = [reserved_fields.link_id, 
              reserved_fields.a_node, 
              reserved_fields.b_node, 
              reserved_fields.direction,
             "time_ab",
             "time_ba"]

dt = [(t, d) for t, d in zip(all_titles, all_types)]

In [ ]:
links = mat.data.shape[0]
k.network = np.zeros(links, dtype=dt)

In [ ]:
k.network[reserved_fields.link_id] = np.arange(links) + 1
k.network[reserved_fields.a_node] = mat.row
k.network[reserved_fields.b_node] = mat.col
k.network[reserved_fields.direction] = np.ones(links)
k.network["time_ab"] = mat.data
k.network["time_ba"] = mat.data * 10000

k.type_loaded = 'NETWORK'
k.status = 'OK'
k.network_ok = True
k.prepare_graph(idx)
del mat

In [ ]:
k.num_links = k.graph.shape[0]

In [ ]:
k.set_graph(cost_field='time', skim_fields=[],block_centroid_flows=False)
hns = SkimResults()
hns.set_cores(2)
hns.prepare(k)

skm2 = NetworkSkimming(k, hns)
skm2.execute()